In [1]:
%pylab inline
import pandas as pd
import numpy as np
import fastai
import torch
import torch.nn.functional as F
import torch.nn as nn
from pathlib import Path
import PIL
import tqdm
import os
import json
import shutil
tqdm.monitor_interval = 0

from fastai.conv_learner import resnet34,resnext101, transforms_top_down, CropType, \
    tfms_from_model, ConvLearner, optim, T, Callback, RandomRotateZoom, to_gpu,to_np
from fastai.dataset import Denormalize, ImageData, FilesNhotArrayDataset, \
    ImageClassifierData, csv_source, parse_csv_labels, split_by_idx, read_dir, \
    FilesIndexArrayDataset, dict_source, FilesArrayDataset, DataLoader, ModelData, ImageData
from fastai.layers import AdaptiveConcatPool2d,Flatten
from torch.nn import BatchNorm1d,Dropout,ReLU,Linear,Sequential,Hardtanh,Softmax
from fastai.column_data import PassthruDataset
from fastai.sgdr import TrainingPhase, DecayType
from lifelines.utils import concordance_index
from collections import defaultdict
from aixtras import *
from bio.liver import build_liver_csv_data
from sklearn.preprocessing import MinMaxScaler



Populating the interactive namespace from numpy and matplotlib


In [2]:
torch.backends.cudnn.benchmark=True
torch.cuda.set_device(0)
torch.cuda.current_device()

torch.backends.cudnn.deterministic = True
torch.manual_seed(7)

devs = [0,1,2,3]

In [3]:
LIVER_PATH = Path('/DATA/BIO/GDC/liver')
LIVER_SAMPLES = LIVER_PATH/"samples"
EXP_PATH = LIVER_PATH/"exp_deep_play"
EXP_MODEL_PATH = EXP_PATH/"models"
TRAIN_DIR = EXP_PATH/"train"
TEST_DIR = EXP_PATH/"test"
CSV_DATA = EXP_PATH/"records.csv"

force_rebuild = False

    
for d in [EXP_PATH, EXP_MODEL_PATH]:
    if not d.exists():
        d.mkdir()

In [4]:
csv_data = build_liver_csv_data(
    LIVER_PATH, 
    EXP_PATH, 
    TRAIN_DIR, 
    TEST_DIR, 
    progress=tqdm.tqdm_notebook,
    force_rebuild=force_rebuild,
    samples_per_patient=50
)

# remember largest possible survival day
t_max = int(csv_data.event_time.max()) # np.int64 will fuck up torch
print(t_max)

dframes = {}
for dsname, df in csv_data.groupby('dsname'):
    dframes[dsname] = df


feat_cols = ['age_at_diagnosis']
n_cols = len(feat_cols)
scaler = MinMaxScaler().fit(dframes['train'][feat_cols].values.astype(float))
for k in dframes:
    dframes[k].loc[:,feat_cols] = scaler.transform(dframes[k].loc[:,feat_cols])

csv data already built
3675


In [5]:
class FilesAndColumnsSurvivalDataset(FilesArrayDataset):
    def __init__(self, fnames, x_cols, y_times_and_observed, t_max, transform, path, jitter=None):
        self.y=y_times_and_observed
        self.t_max = t_max
        self.x_cols = x_cols
        self.n_cols = x_cols.shape[0]
        self.fnames = fnames
        self.jitter = jitter
        super().__init__(fnames, self.y, transform, path)
        assert(len(self.fnames)==len(self.y)==len(self.x_cols))
        self.c = self.get_c()
        
    def get_y(self, i): return self.y[i]
    
    def get_c(self):
        return self.t_max
    
    def get_x(self, i):
        x_img = super().get_x( i)
        x_cols = self.x_cols[i]
        return [x_cols, x_img]
    
    def get1item(self, idx):
        x,y = self.get_x(idx),self.get_y(idx)
        x_cols, x_img = x
        if self.jitter:
            jitter = np.random.randn(*x_cols.shape) * self.jitter + 1.0
            x_cols *= jitter

        x_img, y = self.get(self.transform, x_img, y)
        x_flat = np.concatenate([x_cols.flatten(), x_img.flatten()])
        return x_flat, y


In [6]:
def get_data(sz, bs, tfms, num_workers=8):
    def df_to_ds(key, tfm, jitter=None):
        df = dframes[key]
        ds = FilesAndColumnsSurvivalDataset(
                df.dest_tile.values, 
                df[feat_cols].values, 
                df[['event_time','event_type']].values, 
                t_max, 
                tfm,
                EXP_PATH,
                jitter=jitter
        )
        return ds

    
    datasets = [
        df_to_ds('train', tfms[0], jitter=None), #0.01),
        df_to_ds('valid', tfms[1]),
        df_to_ds('train', tfms[1]),
        df_to_ds('valid', tfms[0], jitter=None), #0.01),
        df_to_ds('test', tfms[1]),
        df_to_ds('test', tfms[0], jitter=None) #0.01)
    ]
    
    classes = range(t_max+1)
    model_data = ImageData(EXP_PATH, datasets, bs, num_workers, classes)
    return model_data


num_evt_types = 1
def custom_loss(preds, target):
    #import pdb; pdb.set_trace()
    evt_times = target[:,0]
    evt_types = target[:,1]
    l1_loss, pairwise_loss = deephit_loss(preds, evt_times, evt_types, t_max+1, num_evt_types)
    b1 = 0.50
    b2 = 0.50
    return b1 * pairwise_loss + b2 * l1_loss 



class ConcordanceIndex(Callback):
    def __init__(self, learner):
        self.learner = learner
        self.best_c_index = -np.Inf
        self.reset()

    def on_epoch_begin(self, metrics):
        self.reset()

    def on_epoch_end(self, metrics):
        ci = concordance_index(
            np.array(self.evt_times), 
            np.array(self.preds), 
            np.array(self.evt_types)
        )
        print('ci: ', ci, len(self.preds), len(self.evt_times))
        if ci>self.best_c_index:
            self.best_c_index = ci
            self.learner.save('best_ci_liver_age_1')
            print('c index improved!')
        self.reset()

    def reset(self):
        self.preds = []
        self.evt_times = []
        self.evt_types = []
        self.mcount = 0
       
    def concordance_metric(self, preds, target):
        #import pdb; pdb.set_trace()
        self.evt_times += list(target[:,0])
        self.evt_types += list(target[:,1])
        self.preds += list(np.argmax(preds, axis=1))
        self.mcount += 1
        return 0.0 #self.mcount    

In [7]:
f_model = resnet34
bs = 64
sz = 256
tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down) 
md = get_data(sz, bs, tfms)


In [8]:
class SplitCols(nn.Module):
    def __init__(self, img_model, cols_model, sz, n_chan, n_cols):
        super().__init__()
        self.sz = sz
        self.n_cols = n_cols
        self.n_chan = n_chan
        self.img_model = img_model
        self.cols_model = cols_model
        self.final = nn.Sequential(
            #nn.Linear(in_features=2*len(md.classes), out_features=len(md.classes)),
            nn.Softmax()
        )

    def forward(self, x):
        bs = x.shape[0]
        x_cols = x[:,0:self.n_cols]
        #x_img = x[:,self.n_cols:].reshape((bs,self.n_chan,sz,sz))
        #img_result = self.img_model(x_img)
        cols_result = cols_model(x_cols)
        #x_combo = torch.cat((cols_result, img_result), dim=1)
        x_combo = cols_result # + img_result
        #import pdb; pdb.set_trace()
        return self.final(x_combo)


feat = 1024

layers = [AdaptiveConcatPool2d(), Flatten()]
layers += [
    BatchNorm1d(feat),
    Dropout(p=0.5), 
    Linear(in_features=feat, out_features=256), 
    ReLU(), 
    BatchNorm1d(256),
    Dropout(p=0.5), 
    Linear(in_features=256, out_features=len(md.classes))
]
custom_head = Sequential(*layers)

hidden = 25
cols_model = nn.Sequential(
    Linear(in_features=n_cols, out_features=hidden), 
    ReLU(), 
    BatchNorm1d(hidden),
    Dropout(p=0.5), 
    Linear(in_features=hidden, out_features=len(md.classes))
)

In [9]:
complete_age_model = to_gpu(SplitCols(None,cols_model,sz,2,n_cols))
print(complete_age_model)

SplitCols(
  (cols_model): Sequential(
    (0): Linear(in_features=1, out_features=25, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=25, out_features=3676, bias=True)
  )
  (final): Sequential(
    (0): Softmax()
  )
)


In [10]:
print(md.trn_dl.batch_size)
print(len(md.trn_dl))
print(64*186)
print(md.trn_dl.dataset.n) #appears that we have 186 batches of 64 size

64
116
11904
7377


In [11]:
# Set the logger
from aixtras.logger import Logger

## inside /bio/liver/notebooks run this:
# $tensorboard --logdir='./logs' --port=6006

## then open up localhost:6006/ and view the stats as they come in

/home/fredmonroe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
## manual train loop allows us to use TensorBoard and to inspect gradients, etc
data_loader = md.trn_dl
criterion = custom_loss
net = complete_age_model
optimizer = torch.optim.Adam(net.parameters())
logger = Logger('./logs/age_fit_logger')


data_iter = iter(data_loader)
iter_per_epoch = len(data_loader)
total_epochs = 5
total_steps = total_epochs*iter_per_epoch

# Start training
for step in range(total_steps):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)

    x,y = next(data_iter)
    
    # Forward, backward and optimize
    optimizer.zero_grad()  # zero the gradient buffer
    outputs = net(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    # Compute accuracy; this probably does not make sense, but I had it from template before...
    _, argmax = torch.max(outputs, 1)
    accuracy = (y[:,0] == argmax.squeeze()).float().mean()

    if (step+1) % 25 == 0:
        print ('Step [%d/%d], Loss: %.4f, Acc: %.2f' 
               %(step+1, total_steps, loss.item(), accuracy.item()))

        #============ TensorBoard logging ============#
        # (1) Log the scalar values
        info = {
            'loss': loss.item(),
            'accuracy': accuracy.item()
        }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

        # (2) Log values and gradients of the parameters (histogram)
        for tag, value in net.named_parameters():
            tag = tag.replace('.', '/')
            logger.histo_summary(tag, to_np(value), step+1)
            logger.histo_summary(tag+'/grad', to_np(value.grad), step+1)


Step [25/580], Loss: 101.0651, Acc: 0.00
Step [50/580], Loss: 71.2140, Acc: 0.00
Step [75/580], Loss: 100.1514, Acc: 0.00
Step [100/580], Loss: 104.8883, Acc: 0.00
Step [125/580], Loss: 36.2603, Acc: 0.00
Step [150/580], Loss: 93.2080, Acc: 0.00
Step [175/580], Loss: 67.4753, Acc: 0.00
Step [200/580], Loss: 28.9697, Acc: 0.00
Step [225/580], Loss: 78.6993, Acc: 0.00
Step [250/580], Loss: 78.7906, Acc: 0.02
Step [275/580], Loss: 97.2018, Acc: 0.02
Step [300/580], Loss: 79.5965, Acc: 0.02
Step [325/580], Loss: 75.0766, Acc: 0.03
Step [350/580], Loss: 118.8995, Acc: 0.02
Step [375/580], Loss: 87.7741, Acc: 0.02
Step [400/580], Loss: 98.8862, Acc: 0.03
Step [425/580], Loss: 27.0166, Acc: 0.02
Step [450/580], Loss: 89.8451, Acc: 0.03
Step [475/580], Loss: 100.9382, Acc: 0.02
Step [500/580], Loss: 116.2073, Acc: 0.05
Step [525/580], Loss: 69.7413, Acc: 0.05
Step [550/580], Loss: 112.3434, Acc: 0.05
Step [575/580], Loss: 31.7113, Acc: 0.03


In [13]:
learn = ConvLearner.pretrained(f_model, md, custom_head=custom_head)
cindex = ConcordanceIndex(learn)
callbacks = [cindex]
learn.crit = custom_loss
learn.metrics = [cindex.concordance_metric]

learn.unfreeze()
learn.models.model = to_gpu(SplitCols(learn.models.model, cols_model, sz, 3, n_cols))